## Part 4: Fraud

In [2]:
%run "../scripts/ETL.py"

The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [6]:
print(fraud_consumer.count())
fraud_consumer.limit(5)

34864


user_id,order_datetime,fraud_probability
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035


Probability of fraud for high revenue merchants...?

In [7]:
print(fraud_merchants.count())
fraud_merchants.limit(5)

114


merchant_abn,order_datetime,fraud_probability
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [10]:
final_join.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id
Felis Limited,10023283211,"furniture, home f...",0.18,e,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,10023283211,74.68707218671928,b36e56d0-dbb2-479...,2021-08-11,23679,6664
Felis Limited,10023283211,"furniture, home f...",0.18,e,Kelly Krueger,039 Green Route,WA,6935,Undisclosed,10023283211,319.4986425058244,51798dc0-8a7b-4cd...,2021-12-28,2199,8509
Felis Limited,10023283211,"furniture, home f...",0.18,e,Brianna Hart,850 Joshua Viaduc...,WA,6415,Female,10023283211,123.61804677878001,edc9c5b1-82a0-4ee...,2021-10-13,19278,15265
Felis Limited,10023283211,"furniture, home f...",0.18,e,Jose Frye,394 Matthews View...,NSW,2021,Male,10023283211,191.63853773963905,376cc026-5d91-498...,2021-07-04,5156,19115
Felis Limited,10023283211,"furniture, home f...",0.18,e,Matthew Waters,729 Harper Spring...,WA,6253,Male,10023283211,250.13111867397248,0220b2de-768c-49e...,2022-07-09,23774,20250


- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [21]:
fj_with_fraud = final_join.join(fraud_consumer, (final_join.user_id == fraud_consumer.user_id) \
                               & (final_join.order_datetime == fraud_consumer.order_datetime), "left_outer")
fj_with_fraud.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,user_id,order_datetime,fraud_probability
Felis Limited,10023283211,"furniture, home f...",0.18,e,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,10023283211,74.68707218671928,b36e56d0-dbb2-479...,2021-08-11,23679,6664,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Kelly Krueger,039 Green Route,WA,6935,Undisclosed,10023283211,319.4986425058244,51798dc0-8a7b-4cd...,2021-12-28,2199,8509,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Brianna Hart,850 Joshua Viaduc...,WA,6415,Female,10023283211,123.61804677878001,edc9c5b1-82a0-4ee...,2021-10-13,19278,15265,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Jose Frye,394 Matthews View...,NSW,2021,Male,10023283211,191.63853773963905,376cc026-5d91-498...,2021-07-04,5156,19115,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Matthew Waters,729 Harper Spring...,WA,6253,Male,10023283211,250.13111867397248,0220b2de-768c-49e...,2022-07-09,23774,20250,null,null,null


In [22]:
fj_rem_fraud = fj_with_fraud.filter("fraud_probability is NULL")
fj_rem_fraud.count()

14115157

In [16]:
user_details.count()

499999

In [23]:
final_join.count()

14195505

In [24]:
tbl_merchants.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels
Felis Limited,10023283211,"furniture, home f...",0.18,e
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",4.22,b
Nunc Sed Company,10165489824,"jewelry, watch, c...",4.4,b
Ultricies Digniss...,10187291046,"wAtch, clock, and...",3.29,b
Enim Condimentum PC,10192359162,music shops - mus...,6.33,a
